---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from collections import deque

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
import re

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. 
    '''
    
    pattern = re.compile(r'''
                     \(.*\)          # match parenths and all chars inside
                    |
                    \[.*\]           # match square brackets and all content
                     ''', re.VERBOSE)

    with open('university_towns.txt') as file:
        # clean off parentheses, square brackets, and trailing whitespace
        lines = (pattern.sub('', line).strip() for line in file)

        values = states.values()
        state, state_regions = None, list()
        for line in lines:
            if line in values:
                state = line
            else:
                entry = (state, line)
                state_regions.append(entry)

    # add to dataframe with ['State'] and ['RegionName'] cols
    college_towns = pd.DataFrame(state_regions, columns=['State', 'RegionName'])

    return college_towns

get_list_of_university_towns()

FileNotFoundError: [Errno 2] No such file or directory: 'university_towns.txt'

In [7]:
def get_house_prices():
    return pd.read_csv('City_Zhvi_AllHomes.csv')
    
get_house_prices()

FileNotFoundError: File b'City_Zhvi_AllHomes.csv' does not exist

In [8]:
def get_gdp():
    '''
    reads excel file of GDP data by year for US. 
    returns a dataframe with index 'Quarter' and 2 columns for GDP.
    '''
    columns = ['GDP in billions of chained 2009 dollars.1']
    gdp = pd.read_excel('gdplev.xls', 
                        index_col=0, 
                        header=0, 
                        skiprows=5).set_index('Unnamed: 4').loc['2000q1':, columns]
    gdp.index.name = 'Quarter'
    return gdp

get_gdp()

FileNotFoundError: [Errno 2] No such file or directory: 'gdplev.xls'

In [9]:
'''
Use a deque
'''

alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

idx4 = 4
deque_of_5 = deque(alphabet[0:idx4+1], maxlen=5)

for i in range(len(alphabet)-4):

    print(deque_of_5)
    idx4 += 1
    try:
        deque_of_5.append(alphabet[idx4])
    except IndexError:
        None

deque(['A', 'B', 'C', 'D', 'E'], maxlen=5)
deque(['B', 'C', 'D', 'E', 'F'], maxlen=5)
deque(['C', 'D', 'E', 'F', 'G'], maxlen=5)
deque(['D', 'E', 'F', 'G', 'H'], maxlen=5)
deque(['E', 'F', 'G', 'H', 'I'], maxlen=5)
deque(['F', 'G', 'H', 'I', 'J'], maxlen=5)
deque(['G', 'H', 'I', 'J', 'K'], maxlen=5)
deque(['H', 'I', 'J', 'K', 'L'], maxlen=5)
deque(['I', 'J', 'K', 'L', 'M'], maxlen=5)
deque(['J', 'K', 'L', 'M', 'N'], maxlen=5)
deque(['K', 'L', 'M', 'N', 'O'], maxlen=5)
deque(['L', 'M', 'N', 'O', 'P'], maxlen=5)
deque(['M', 'N', 'O', 'P', 'Q'], maxlen=5)
deque(['N', 'O', 'P', 'Q', 'R'], maxlen=5)
deque(['O', 'P', 'Q', 'R', 'S'], maxlen=5)
deque(['P', 'Q', 'R', 'S', 'T'], maxlen=5)
deque(['Q', 'R', 'S', 'T', 'U'], maxlen=5)
deque(['R', 'S', 'T', 'U', 'V'], maxlen=5)
deque(['S', 'T', 'U', 'V', 'W'], maxlen=5)
deque(['T', 'U', 'V', 'W', 'X'], maxlen=5)
deque(['U', 'V', 'W', 'X', 'Y'], maxlen=5)
deque(['V', 'W', 'X', 'Y', 'Z'], maxlen=5)


In [1]:
def check_recession(window):
    start, descent, mid, ascent, end = window
    descending = start[1] > descent[1]
    valley = descent[1] > mid[1] < ascent[1]
    ascending = ascent[1] < end[1]
    
    if all([descending, valley, ascending]):
        return True
    else:
        return False

In [2]:
from pprint import pprint

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = get_gdp()
    
    iterator = df.itertuples()
    
    scope = deque(maxlen=5)
    for i in range(5):
        scope.append(next(iterator))
    check_recession(scope)
#     for row in iterator:
#         scope.append(row)
        
get_recession_start()

NameError: name 'get_gdp' is not defined

In [ ]:
def get_recession():
    
    df = get_gdp()
    recession_start = list()
    
    four_qs = [(quarter, gdp) for quarter, gdp in df.iloc[0:4].itertuples()]
    
    if (four_qs[0][1]>four_qs[1][1]) & (four_qs[2][1]<four_qs[3][1]):
        recession_start.append(four_qs[0][0])
    else:
        four_qs.pop(0)
        q, gdp = df.itertuples().next()
    return row

get_recession()

In [ ]:
df = get_gdp()

next(df.itertuples())
q, gdp = df.itertuples()

# for i in range(df.shape[0]):
#     if (next(gdp)>next(gdp)):
#         print('Decline!')
#         if (next(gdp)<next(gdp)):
#             print('Recession!')
#     else:
#         print('Not a recession')


In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return "ANSWER"

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return "ANSWER"

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"